In [ ]:
# %%capture
# pip install mne

!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.6/281.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
# import mne
import pywt
import wandb

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DWT_file = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/DWT/DWT.npz'
DWT = np.load(DWT_file, allow_pickle=True)  # Allow loading pickled object arrays
X = DWT['X'].astype('float64')
Y = DWT['Y'].astype('float64')
group = DWT['G'].astype('float64')
X= np.moveaxis(X,1,2)
X.shape

(5834, 52, 29)

#Split the data

In [ ]:
X_reshaped = X.reshape(X.shape[0],-1)
X_reshaped.shape



#Scale data
scaler = StandardScaler()
scaled_data_reshaped = scaler.fit_transform(X_reshaped)

# Reshape back to original shape if necessary
scaled_data = scaled_data_reshaped.reshape(X.shape)


In [ ]:
# Configuration
n_splits = 5  # Number of folds
random_state = 42
shuffle = True

# Initialize KFold
kf = KFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

# Create a placeholder array for fold assignments
fold_assignments = np.empty(len(Y), dtype=int)

# Split the data using kf.split. Note: kf.split does not need Y for splitting, but it's kept here for consistency
for fold, (train_idx, test_idx) in enumerate(kf.split(scaled_data, Y)):
    # Assign fold number
    fold_assignments[test_idx] = fold

# Example: To get the train and test sets for the first fold
fold_number = 0  # Choose which fold to use (0 to n_splits-1)
train_idx = np.where(fold_assignments != fold_number)[0]
test_idx = np.where(fold_assignments == fold_number)[0]

X_train, X_test = scaled_data[train_idx], scaled_data[test_idx]
Y_train, Y_test = Y[train_idx], Y[test_idx]

In [ ]:
X_train.shape,Y_train.shape,X_test.shape

((4667, 52, 29), (4667,), (1167, 52, 29))

#CNN Model

In [ ]:

!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


This is with using only the data with out any transformations applied. So This is just plain Deep learning.

In [ ]:
X_train.shape, Y_train.shape,X_test.shape

((4667, 52, 29), (4667,), (1167, 52, 29))

In [ ]:
from wandb.keras import WandbCallback

ModuleNotFoundError: No module named 'wandb.keras'

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import callbacks
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import  Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,Flatten,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D,ReLU,Conv2D,MaxPool2D,GlobalAveragePooling2D,AveragePooling2D,Bidirectional, LSTM, concatenate
from tensorflow.keras.regularizers import l2
from keras.layers import ConvLSTM1D, LSTM

from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM

# early_stopping = callbacks.EarlyStopping(
#     min_delta=0.001,
#     patience=10,
#     restore_best_weights=True
# )

from keras import backend as K

def focal_loss(alpha=0.25,gamma=1.5):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)

        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))

        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from keras_radam import RAdam

checkpoint = ModelCheckpoint("/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Mobilenet1.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = .25,
                              patience = 2,
                              verbose = 1)


# tf_callback = tf.keras.callbacks.TensorBoard(
#                             log_dir="/content/drive/MyDrive/DeepLearning/TomatoLeaf/logs",
#                             histogram_freq=1,
#                             write_graph=True,
#                             write_images=True,
#                             update_freq="epoch"
#                             )

# we put our call backs into a callback list
wandb.init(project="Epileptic Seizure", entity="RaihanRabby", dir = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/wandb')
callbacks = [checkpoint,reduce_lr,WandbCallback()]

wandb: Currently logged in as: fazlarabby-raihan45. Use `wandb login --relogin` to force relogin


NameError: name 'WandbCallback' is not defined

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Conv1DTranspose, Flatten, Dense, Reshape, Cropping1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import RMSprop, Adam
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)


input_shape = (52,29)
# Define the autoencoder model
Encoder = Sequential([
    Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape),
    BatchNormalization(),
    MaxPooling1D(2, padding="same"),
    Conv1D(64, 3, strides=1, activation='relu', padding="same"),
    BatchNormalization(),
    MaxPooling1D(2, padding="same"),
    Conv1D(29, 3, strides=1, activation='relu', padding="same"),
    BatchNormalization(),
    MaxPooling1D(2, padding="same"),

    Conv1DTranspose(29, 3, strides=2, activation='relu', padding="same"),
    BatchNormalization(),
    Conv1DTranspose(64, 3, strides=2, activation='relu', padding="same"),
    BatchNormalization(),
    Conv1DTranspose(256, 3, strides=2, activation='relu', padding="same"),
    BatchNormalization(),
    Cropping1D(cropping=(2, 2)),
    Flatten(),
    Dense(np.prod(input_shape), activation='sigmoid'),  # Ensures output size matches input
    Reshape(input_shape)  # Reshapes output to desired input shape
])
opt = Adam(learning_rate=.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# Compile the autoencoder
Encoder.compile(opt, loss='mse')
Encoder.summary()

In [ ]:
Encoder.fit(X_train, X_train, epochs=10, batch_size=128, validation_data=(X_test, X_test),callbacks=[early_stopping, model_checkpoint])

NameError: name 'Encoder' is not defined

In [ ]:
from keras.models import Model

# Define the encoder as being the first part of the autoencoder
encoder_output = Encoder.layers[8].output  # -6 should be the last MaxPooling1D layer if counting backwards
encoder_model = Model(inputs=Encoder.input, outputs=encoder_output)

# Summary to confirm model structure
encoder_model.summary()

NameError: name 'Encoder' is not defined

In [ ]:
encoded_X_train = encoder_model.predict(X_train)
encoded_X_test = encoder_model.predict(X_test)

NameError: name 'encoder_model' is not defined

In [ ]:
%%time
from keras.optimizers import RMSprop, Adam
from tensorflow.keras.regularizers import L2

# Reshape data for LSTM or Conv1D
# X_train = X_train.reshape((-1, 1508, 1))
# X_test = X_test.reshape((-1, 1508, 1))
# from tensorflow.keras.layers import Conv1D, Conv1DTranspose, MaxPool1D, UpSampling1D, Flatten, Dense, Reshape, ReLU, Dropout, LeakyReLU
# Now, your model architecture could correctly use LSTM or Conv1D layers
def build_sequential_model(input_shape):

    clear_session()
    model = Sequential()


    # Example with Conv1D
    model.add(Conv1D(filters=64, kernel_size=2,strides =1, activation='relu', input_shape=input_shape))
    # model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    # # model.add(MaxPool1D(pool_size=4))
    # model.add(Conv1D(filters=256, kernel_size=5, activation='relu'))
    # model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    # model.add(Conv1D(filters=64, kernel_size=5,strides =1, activation='relu'))

    # model.add(MaxPool1D(pool_size=4,padding = 'same'))
    # model.add(MaxPool1D(pool_size=4,padding = 'same'))
    model.add(MaxPool1D(pool_size=2,padding = 'same'))


    model.add(LSTM(128, return_sequences=True, )),
    model.add(LSTM(32, return_sequences=False))

    model.add(Dropout(0.25))


    # Flatten for Dense layers after Conv1D/LSTM processing
    model.add(Flatten())
    model.add(Dense(1024, activation='relu', kernel_regularizer = L2(0.001)))
    model.add(Dense(512, activation='relu', kernel_regularizer = L2(0.001)))
    model.add(Dense(128, activation='relu', kernel_regularizer = L2(0.001)))
    model.add(Dense(1, activation='sigmoid'))

    opt = Adam(learning_rate=.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(optimizer = opt,
        loss=focal_loss(),
        metrics=['accuracy'])
    return model

Model = build_sequential_model(input_shape=(52,29))
Model.fit(encoded_X_train, Y_train, epochs=50, batch_size=32, callbacks=[callbacks], validation_data = (encoded_X_test, Y_test), shuffle=True)

NameError: name 'encoded_X_train' is not defined

37/37 [==============================] - 1s 24ms/step
